In [132]:
import os
import fnmatch
import json
import dateutil.parser
import re
import pprint
import glob
import pandas as pd

In [175]:
with open("/treehouse/treeshop/manifests/140.1.txt") as f:
    ids = sorted([word.strip() for line in f.readlines() for word in line.split(',') if word.strip()])
print("Found {} ids".format(len(ids)))

Found 4 ids


In [186]:
# All methods
methods = [json.loads(open(m).read()) for id in ids 
           for m in glob.glob("/treehouse/archive/downstream/{}/secondary/*/methods.json".format(id))]
print("Found {} secondary outputs".format(len(methods)))

Found 20 secondary outputs


In [187]:
# UMEND QC
reports = [{"id": id, **json.loads(open(m).read())} for id in ids 
           for m in glob.glob("/treehouse/archive/downstream/{}/secondary/*bam-umend-qc-1.1.0-cc481e4/bam_umend_qc.json".format(id))]
print("{} Completed RNASeq and BAM UMEND QC".format(len(reports)))
print("{} Passed".format(sum(r["qc"] == "PASS" for r in reports)))
print("{} Failed: {}".format(sum(r["qc"] == "FAIL" for r in reports), [r["id"] for r in reports if r["qc"] == "FAIL"]))

4 Completed RNASeq and BAM UMEND QC
3 Passed
1 Failed: ['TH01_0717_S01']


In [190]:
pd.DataFrame(reports).set_index('id')

,estExonicUniqMappedNonDupeReadCount,input,qc,uniqMappedNonDupeReadCount
id,,,,
TH01_0710_S01,4.469763e+07,readDist.txt,PASS,64867826.0
TH01_0713_S01,3.397506e+07,readDist.txt,PASS,71991565.0
TH01_0715_S01,4.137297e+07,readDist.txt,PASS,55923263.5
TH01_0717_S01,9.226395e+06,readDist.txt,FAIL,19771111.0


In [191]:
[(r["id"], r["qc"], r["estExonicUniqMappedNonDupeReadCount"]) for r in reports]

[('TH01_0710_S01', 'PASS', 44697633),
 ('TH01_0713_S01', 'PASS', 33975062.7),
 ('TH01_0715_S01', 'PASS', 41372967.375),
 ('TH01_0717_S01', 'FAIL', 9226394.94)]

In [183]:
reports = [{"id": id, 
            "methods": [m for m in glob.glob("/treehouse/archive/downstream/{}/secondary/*/methods.json".format(id))]}
            for id in ids]
[(r["id"], len(r["methods"])) for r in reports]

[('TH01_0710_S01', 2),
 ('TH01_0713_S01', 5),
 ('TH01_0715_S01', 5),
 ('TH01_0717_S01', 5)]

In [173]:
runtimes = pd.DataFrame([{"id": m["sample_id"], 
               "pipeline": m["pipeline"]["docker"]["url"],
               "duration": (dateutil.parser.parse(m["end"]) 
                            - dateutil.parser.parse(m["start"])).total_seconds()/(60*60)}
 for m in methods])
runtimes

,duration,id,pipeline
0,0.020491,TH01_0710_S01,https://hub.docker.com/alpine
1,0.011007,TH01_0713_S01,https://hub.docker.com/alpine
2,7.761169,TH01_0713_S01,https://quay.io/ucsc_cgl/rnaseq-cgl-pipeline
3,0.011527,TH01_0715_S01,https://hub.docker.com/alpine
4,7.843065,TH01_0715_S01,https://quay.io/ucsc_cgl/rnaseq-cgl-pipeline
5,0.006318,TH01_0717_S01,https://hub.docker.com/alpine
6,4.231748,TH01_0717_S01,https://quay.io/ucsc_cgl/rnaseq-cgl-pipeline
7,4.446321,TH01_0717_S01,https://hub.docker.com/r/ucsctreehouse/bam-ume...
8,1.843002,TH01_0717_S01,https://hub.docker.com/r/ucsctreehouse/fusion
9,0.146709,TH01_0717_S01,https://hub.docker.com/r/ucsctreehouse/mini-va...


In [159]:
stats = runtimes.pivot(columns="pipeline").describe()
stats

duration  \
pipeline https://hub.docker.com/alpine   
count                        12.000000   
mean                          0.023692   
std                           0.006138   
min                           0.018981   
25%                           0.019590   
50%                           0.021123   
75%                           0.024795   
max                           0.038409   

                                                              \
pipeline https://hub.docker.com/r/ucsctreehouse/bam-umend-qc   
count                                             12.000000    
mean                                               6.956021    
std                                                1.688410    
min                                                5.237647    
25%                                                5.626776    
50%                                                6.219239    
75%                                                8.177105    
max                                               10.152545    

                                                        \
pipeline https://hub.docker.com/r/ucsctreehouse/fusion   
count                                        12.000000   
mean                                          2.934652   
std                                           0.963998   
min                                           2.112196   
25%                                           2.288067   
50%                                           2.636636   
75%                                           3.077107   
max                                           5.494860   

                                                               \
pipeline https://hub.docker.com/r/ucsctreehouse/mini-var-call   
count                                             12.000000     
mean                                               0.214092     
std                                                0.043735     
min                                                0.154571     
25%                                                0.186933     
50%                                                0.216102     
75%                                                0.233472     
max                                                0.302643     

                                                       
pipeline https://quay.io/ucsc_cgl/rnaseq-cgl-pipeline  
count                                       12.000000  
mean                                        20.239206  
std                                          4.083669  
min                                         15.158212  
25%                                         17.298667  
50%                                         19.556427  
75%                                         22.941253  
max                                         27.597463

In [162]:
stats.sum(axis=1)

count    60.000000
mean     30.367663
std       6.785950
min      22.681608
25%      25.420033
50%      28.649527
75%      34.453732
max      43.585920
dtype: float64

In [160]:
reports = [{"id": id, **json.loads(open(m).read())} for id in ids 
           for m in glob.glob("/treehouse/archive/downstream/{}/secondary/*fusion*/methods.json".format(id))]